In [13]:
import requests
import time

# Définition de l'User-Agent pour éviter le blocage
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}

# Liste des produits à scraper
products = ['smartphone', 'Smartwatch', 'ecouteurs', 'PC']

# URLs des sites de vente en ligne
base_urls = {
    'Amazon': 'https://www.amazon.fr/s?k={}',
    'AliExpress': 'https://www.aliexpress.com/wholesale?SearchText={}',
    'Boulanger': 'https://www.boulanger.com/resultats?tr={}',
    'LDLC': 'https://www.ldlc.com/recherche/{}',
    'Materiel.net': 'https://www.materiel.net/recherche/{}',
    'Banggood': 'https://www.banggood.com/search/{}.html'
}

# Fonction de scraping
def scrape_product(product):
    print(f"Scraping {product}...")
    for site, base_url in base_urls.items():
        url = base_url.format(product.replace(' ', '+'))
        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()  # Vérification des erreurs HTTP
            print(f"✅ {site} : Données récupérées pour {product}.")
        except requests.exceptions.RequestException as e:
            print(f"❌ Erreur lors de la requête {site} : {e}")

        # Pause pour éviter le blocage par les sites
        time.sleep(2)

# Exécution du scraping pour tous les produits
for product in products:
    scrape_product(product)


Scraping smartphone...
✅ Amazon : Données récupérées pour smartphone.
✅ AliExpress : Données récupérées pour smartphone.
✅ Boulanger : Données récupérées pour smartphone.
✅ LDLC : Données récupérées pour smartphone.
✅ Materiel.net : Données récupérées pour smartphone.
✅ Banggood : Données récupérées pour smartphone.
Scraping Smartwatch...
✅ Amazon : Données récupérées pour Smartwatch.
✅ AliExpress : Données récupérées pour Smartwatch.
✅ Boulanger : Données récupérées pour Smartwatch.
✅ LDLC : Données récupérées pour Smartwatch.
✅ Materiel.net : Données récupérées pour Smartwatch.
✅ Banggood : Données récupérées pour Smartwatch.
Scraping ecouteurs...
✅ Amazon : Données récupérées pour ecouteurs.
✅ AliExpress : Données récupérées pour ecouteurs.
✅ Boulanger : Données récupérées pour ecouteurs.
✅ LDLC : Données récupérées pour ecouteurs.
✅ Materiel.net : Données récupérées pour ecouteurs.
✅ Banggood : Données récupérées pour ecouteurs.
Scraping PC...
✅ Amazon : Données récupérées pour PC.

In [ ]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from parsel import Selector

# Définition de l'User-Agent
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}

# Liste des sites à scraper
base_urls = {
    'Boulanger': 'https://www.boulanger.com/resultats?tr={}',
}

# Liste des produits à scraper
products = ['smartphone', 'smartwatch', 'ecouteurs', 'PC']

# Stockage des résultats
all_data = []

# Fonction de scraping
def scrape_product(site, url, product):
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()  # Vérification des erreurs HTTP
        soup = BeautifulSoup(response.content, 'html.parser')
#\39 3e9ff98-ddd5-4940-ad69-a22ec4f875e6 > div > div > div > div > span > div > div > div.a-section.a-spacing-small.puis-padding-left-small.puis-padding-right-small > div.a-section.a-spacing-none.a-spacing-top-small.s-price-instructions-style
        # Extraction des données selon le site
        if site == "Boulanger":
            items = soup.select('#product-list > ul > li > article')
            for item in items:
                title = item.select_one('#productLabel')
                price= item.select_one('div.product-list__product-area-3.g-col-5.g-col-sm-7.g-col-md-4.g-start-md-9.g-col-lg-3.g-start-lg-7.g-col-xl-3.g-start-xl-7 > div > p')
                rating = item.select_one('div.product-list__product-area-2.g-col-5.g-col-sm-7.g-col-md-4.g-col-lg-3.g-col-xl-3 > div.product-list__product-rating > a > div > bl-rating').get('rating')
                reviews = item.select_one('div.product-list__product-area-2.g-col-5.g-col-sm-7.g-col-md-4.g-col-lg-3.g-col-xl-3 > div.product-list__product-rating > a > span')
                stock = item.select_one('div.product-list__product-area-3.g-col-5.g-col-sm-7.g-col-md-4.g-start-md-9.g-col-lg-3.g-start-lg-7.g-col-xl-3.g-start-xl-7 > div > div > span.price__crossed')
                reference = item.get("data-product-id", "N/A")

                if title and price:
                    all_data.append({
                        'Produit': product,
                        'Site': site,
                        'Titre': title.text.strip(),
                        'Prix (€)': price.text.replace(',', '.').strip() if price else None,
                        'Note': rating if rating else None,
                        'Avis': reviews if reviews else None,  # Boulanger ne montre pas toujours le nombre d’avis
                        'promo': stock.text.strip() if stock else "Indisponible",
                        'Référence': reference
                    })

        # Ajout d'autres sites ici (LDLC, Materiel.net, Banggood) en suivant la même logique

    except requests.exceptions.RequestException as e:
        print(f"❌ Erreur lors de la requête {site}: {e}")

    time.sleep(2)  # Pause pour éviter le blocage

# Exécution du scraping
for product in products:
    for site, base_url in base_urls.items():
        scrape_product(site, base_url.format(product.replace(' ', '+')), product)

# Conversion en DataFrame Pandas pour visualisation
df = pd.DataFrame(all_data)

# Affichage des résultats
print("\n📊 Résultats du scraping :")
print(df)

# Sauvegarde en CSV
df.to_csv("resultats_scrapingboul.csv", index=False)



📊 Résultats du scraping :
      Produit       Site                                              Titre  \
0  smartphone  Boulanger  Smartphone\n\t\t\t\t\t\t\tSAMSUNG\n\t\t\t\t\t\...   
1  smartwatch  Boulanger  Montre connectée\n\t\t\t\t\t\t\tSAMSUNG\n\t\t\...   
2   ecouteurs  Boulanger  Ecouteurs\n\t\t\t\t\t\t\tSAMSUNG\n\t\t\t\t\t\t...   
3          PC  Boulanger  PC Gamer\n\t\t\t\t\t\t\tASUS\n\t\t\t\t\t\t\tTU...   

   Prix (€)               Note    Avis         Stock Référence  
0   421.99€                5.0   [(1)]       381,99€       N/A  
1   699.99€  4.622000217437744  [(82)]       569,00€       N/A  
2    59.99€  4.684199810028076  [(19)]  Indisponible       N/A  
3  1049.99€                5.0   [(6)]  Indisponible       N/A  


In [5]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.common.exceptions import NoSuchElementException
# Configuration de Selenium avec Edgefrom selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

edge_options = Options()
edge_options.add_argument("--headless")  # Exécuter en mode headless (sans interface graphique)
edge_options.add_argument("--disable-gpu")
edge_options.add_argument("--window-size=1920,1080")

# Chemin vers le pilote Edge (Microsoft Edge WebDriver)
edge_service = Service(executable_path="msedgedriver.exe")  # Remplacez par le chemin de votre WebDriver

# Initialisation du navigateur Edge
loading_time = 30

driver = webdriver.Edge(service=EdgeService(EdgeChromiumDriverManager().install()))
   

# Liste des URLs à scraper
urls = [
    'https://www.boulanger.com/c/montre-connectee',
    'https://www.boulanger.com/c/airpods',
    'https://www.boulanger.com/c/tous-les-ordinateurs-portables',
    'https://www.boulanger.com/c/smartphone-telephone-portable'
]

# Liste des produits à scraper
products = ['smartwatch', 'ecouteurs', 'PC', 'smartphone']

# Stockage des résultats
all_data = []

# Fonction de scraping avec Selenium
def scrape_product(url, product):
    try:
        # Charger la page
        driver.get(url)
        time.sleep(5)  # Attendre que la page se charge complètement

        # Extraire les éléments de la page
        items = driver.find_elements(By.CSS_SELECTOR, '#product-list > ul > li > article')  # Sélecteur principal pour les produits
        
        for item in items:
            try:
                title = item.find_element(By.CSS_SELECTOR, '#productLabel').text  # Titre du produit
            except NoSuchElementException:
                title = None

            try:
                price = item.find_element(By.CSS_SELECTOR, '.product-list__product-area-3.g-col-5.g-col-sm-7.g-col-md-4.g-start-md-9.g-col-lg-3.g-start-lg-7.g-col-xl-3.g-start-xl-7 > div > p').text  # Prix
            except NoSuchElementException:
                price = None

            try:
                rating = item.find_element(By.CSS_SELECTOR, 'bl-rating').get_attribute('rating')  # Note
            except NoSuchElementException:
                rating = None

            try:
                reviews = item.find_element(By.CSS_SELECTOR, '.product-list__product-area-2.g-col-5.g-col-sm-7.g-col-md-4.g-col-lg-3.g-col-xl-3 > div.product-list__product-rating > a > span').text  # Avis
            except NoSuchElementException:
                reviews = None

            try:
                promo = item.find_element(By.CSS_SELECTOR, '.product-list__product-area-3.g-col-5.g-col-sm-7.g-col-md-4.g-start-md-9.g-col-lg-3.g-start-lg-7.g-col-xl-3.g-start-xl-7 > div > div > span.price__crossed').text  # Stock ou promo
            except NoSuchElementException:
                promo = "rien"
            try:
                stock = item.find_element(By.CSS_SELECTOR, '.product-list__product-area-3.g-col-5.g-col-sm-7.g-col-md-4.g-start-md-9.g-col-lg-3.g-start-lg-7.g-col-xl-3.g-start-xl-7 > button > span').text  # Stock ou promo
            except NoSuchElementException:
                stock = "Indisponible"

            # Ajouter les données extraites à la liste
            if title and price:
                all_data.append({
                    'Produit': product,
                    'Site': 'Boulanger',
                    'Titre': title.strip(),
                    'Prix (€)': price.replace('€', '').strip() if price else None,
                    'Note': rating if rating else None,
                    'Avis': reviews.strip() if reviews else None,
                    'Promo': promo.replace('€', '').strip() if promo else None,
                    'Stock': stock.strip() if stock else "Indisponible"
                })

    except Exception as e:
        print(f"❌ Erreur lors du scraping de {url} : {e}")

# Exécution du scraping
for product, url in zip(products, urls):
    scrape_product(url, product)

# Fermer le navigateur
driver.quit()

# Conversion en DataFrame Pandas pour visualisation
df = pd.DataFrame(all_data)

# Affichage des résultats
print("\n📊 Résultats du scraping :")
print(df)

# Sauvegarde en CSV
df.to_csv("resultats_scraping_boulanger_selenium.csv", index=False)


📊 Résultats du scraping :
Empty DataFrame
Columns: []
Index: []
